In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
from matplotlib.patches import Rectangle
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

file_path='2025_0001_0037_Detailed_kp.xlsx'
file_path2='2025_0038_0074_Detailed_kp.xlsx'
# Define the puit dictionary
puit_dict = {
    'puit1': ['2025-0001', '2025-0016', '2025-0030', '2025-0044', '2025-0059'],
    'puit2': ['2025-0002', '2025-0017', '2025-0031', '2025-0045', '2025-0060'],
    'puit3': ['2025-0003', '2025-0018', '2025-0032', '2025-0046', '2025-0061'],
    'puit4': ['2025-0004', '2025-0019', '2025-0033', '2025-0047', '2025-0062'],
    'puit5': ['2025-0005', '2025-0020', '2025-0034', '2025-0049', '2025-0063'],
    'puit6': ['2025-0006', '2025-0021', '2025-0035', '2025-0050', '2025-0064'],
    'puit7': ['2025-0007', '2025-0022', '2025-0036', '2025-0051', '2025-0065'],
    'puit8': ['2025-0008', '2025-0023', '2025-0037', '2025-0052', '2025-0066'],
    'puit9': ['2025-0009', '2025-0025', '2025-0038', '2025-0053', '2025-0067'],
    'puit10': ['2025-0010', '2025-0026', '2025-0039', '2025-0054', '2025-0068'],
    'puit11': ['2025-0011', '2025-0027', '2025-0041', '2025-0055', '2025-0069'],
    'puit12': ['2025-0012', '2025-0028', '2025-0042', '2025-0056', '2025-0070'],
    'puit13': ['2025-0029', '2025-0043', '2025-0057', '2025-0071'],
    'blank':['2025-0015'],
    'enzyme':['2025-0058','2025-0074']
}
display_names = [0,7,14,21,28]
# --- Widgets ---
dropdown = widgets.Dropdown(
    options=list(puit_dict.keys()),
    description="Choisir un puit:",
    style={'description_width': 'initial'}
)

summary_toggle = widgets.Checkbox(
    value=False,
    description="Afficher sommaire ?",
    indent=False
)

vk_toggle = widgets.Checkbox(
    value=False,
    description="Afficher Van Krevelen ?",
    indent=False
)

ab_toggle = widgets.Checkbox(
    value=False,
    description="Afficher Abondance Totale ?",
    indent=False
)

classab_toggle = widgets.Checkbox(
    value=False,
    description="Afficher Classe Abondance ?",
    indent=False
)

venclass_toggle = widgets.Checkbox(
    value=False,
    description="Afficher Classe Van Krevelen ?",
    indent=False
)
output = widgets.Output()

def on_puit_change(selected_puit, show_vk, show_summary, show_ab,class_ab,vkclass):
    with output:
        clear_output()
        # --- Chargement et concaténation ---
        sheet_list = puit_dict[selected_puit]
        dfs = []
        for name, day in zip(sheet_list, display_names):
            try:
                df = pd.read_excel(file_path, sheet_name=name, skiprows=2)
            except ValueError:
                try:
                    df = pd.read_excel(file_path2, sheet_name=name, skiprows=2)
                except ValueError:
                    continue
            df["Day"] = day
            dfs.append(df)
        if not dfs:
            print("Aucune donnée chargée.")
            return
        combined_df = pd.concat(dfs, ignore_index=True)

        # --- Calcul du sommaire ---
        # (identique à ton code précédent)
        formulas_per_day = combined_df.groupby("Day")["Formula"].apply(set)
        common = set.intersection(*formulas_per_day)
        trends = []
        for f in common:
            sub = combined_df[combined_df["Formula"]==f]
            pivot = sub.pivot_table(index="Day", values="Total Abund")
            pivot.columns = [f]
            trends.append(pivot)
        df_trends = pd.concat(trends, axis=1).fillna(0).sort_index()
        summary_df = pd.DataFrame(index=df_trends.columns)
        summary_df["Day0"] = df_trends.loc[0]
        summary_df["Day28"] = df_trends.iloc[-1]
        summary_df["FoldChange_28vs0"] = (summary_df["Day28"]/summary_df["Day0"]).replace([float("inf"), -float("inf")], None)
        summary_df["AUC"]       = df_trends.sum()
        summary_df["PeakDay"]   = df_trends.idxmax()
        summary_df["MinDay"]    = df_trends.idxmin()
        def classify(r):
            if r["FoldChange_28vs0"] > 2:    return "Growing"
            if r["FoldChange_28vs0"] < 0.5:  return "Decreasing"
            if r["PeakDay"] in [7,14]:       return "Transient"
            return "Stable"
        summary_df["Behavior"] = summary_df.apply(classify, axis=1)
        meta_cols = ["Formula","Class","DBE","#C","#H","#N","#O","#S","Z Number","H/C","O/C","S/C"]
        meta = combined_df[meta_cols].drop_duplicates("Formula").set_index("Formula")
        summary_full = summary_df.join(meta, how="left")

        # --- Affichage du sommaire si demandé ---
        if show_summary:
            display(summary_full.round(2))

        # --- Van Krevelen si demandé ---
        if show_vk:
            abund_by_day = combined_df.groupby(["Formula","Day"])["Total Abund"].sum().reset_index()
            vk = abund_by_day.merge(summary_full, left_on="Formula", right_index=True).dropna(subset=["H/C","O/C"])
            vk["Day"] = vk["Day"].astype(int)
            vk = vk.sort_values("Day")
            days = sorted(vk["Day"].unique())
            cols = 3
            rows = int((len(days)+1+cols-1)//cols)
            fig, axes = plt.subplots(rows, cols, figsize=(cols*5, rows*4), sharex=True, sharey=True)
            axes = axes.flatten()
            for i, d in enumerate(days):
                ax = axes[i]
                df_d = vk[vk["Day"]==d]
                # zone PAH
                ax.add_patch(Rectangle((0.0,0.4),0.05,0.4, edgecolor="red",facecolor="red",alpha=0.2,
                                       label="PAH Zone" if i==0 else None))
                sc = ax.scatter(df_d["O/C"], df_d["H/C"], c=df_d["DBE"],
                                s=df_d["Total Abund"]/500, alpha=0.7, edgecolors="k")
                ax.set_title(f"Day {d}")
                ax.set_xlabel("O/C"); ax.set_ylabel("H/C"); ax.grid(True)
            # colorbar et légende tailles
            cax = axes[len(days)]; cax.axis("off")
            cb = fig.colorbar(sc, ax=cax, orientation="vertical", fraction=0.8)
            cb.set_label("DBE")
            for sz in [10000,50000,100000]:
                cax.scatter([],[],s=sz/500,label=str(sz),color="gray",edgecolor="k",alpha=0.7)
            cax.legend(title="Total Abund", loc="lower center", frameon=False)
            # suppression des axes vides
            for ax in axes[len(days)+1:]:
                fig.delaxes(ax)
            fig.suptitle(f"Van Krevelen: {selected_puit}", fontsize=16)
            plt.tight_layout(rect=[0,0,1,0.95])
            plt.show()
        if show_ab:
            # --- Affichage de l'abondance totale par jour ---
            fig, ax = plt.subplots(figsize=(10, 6))
            combined_df.groupby("Day")["Total Abund"].sum().plot(ax=ax, marker='o')
            ax.set_title(f"Total Abundance by Day for {selected_puit}")
            ax.set_xlabel("Day")
            ax.set_ylabel("Total Abundance")
            ax.grid(True)
            plt.show()
        if class_ab:
            agg = combined_df.groupby(["Day", "Class"])["Total Abund"].sum().reset_index()
            plt.figure(figsize=(12, 6))
            sns.lineplot(data=agg, x="Day", y="Total Abund", hue="Class", marker="o")
            plt.title(f"Class Abundance Over Time for {selected_puit}")
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.legend(loc='best')
            plt.show()
        if vkclass:
            # === 1. Van Krevelen zone classification ===
            def assign_van_krevelen_group(row):
                h_c, o_c = row["H/C"], row["O/C"]
                if h_c > 1.5 and o_c < 0.3:
                    return "Lipid-like"
                elif h_c > 1.5 and o_c > 0.6:
                    return "Carbohydrate-like"
                elif 1.0 <= h_c <= 1.5 and 0.2 <= o_c <= 0.6:
                    return "Lignin-like"
                elif h_c < 1.0 and o_c < 0.3:
                    return "Condensed Aromatic"
                elif o_c > 0.6:
                    return "Tannin-like"
                else:
                    return "Other"

            summary_full["VK Group"] = summary_full.apply(assign_van_krevelen_group, axis=1)

            # === 2. Prepare abundance matrix for PCA ===
            # df_trends: index=Day, columns=Formulas
            X = df_trends.T.values  # Formulas x Timepoints
            X_scaled = StandardScaler().fit_transform(X)  # normalize

            # === 3. PCA ===
            pca = PCA(n_components=2)
            X_pca = pca.fit_transform(X_scaled)

            # Create PCA dataframe
            pca_df = pd.DataFrame(X_pca, columns=["PC1", "PC2"], index=df_trends.columns)
            pca_df["VK Group"] = summary_full["VK Group"]
            pca_df["Behavior"] = summary_full["Behavior"]

            # === 4. KMeans clustering (optional: change n_clusters) ===
            kmeans = KMeans(n_clusters=4, random_state=42, n_init="auto")
            pca_df["Cluster"] = kmeans.fit_predict(X_scaled)

            # === 5. Plot PCA by cluster ===
            plt.figure(figsize=(10, 7))
            sns.scatterplot(data=pca_df, x="PC1", y="PC2", hue="Cluster", palette="Set2", style="VK Group", s=100, edgecolor="k")
            plt.title(f"PCA of Temporal Abundance Trends (Clustered) for {selected_puit}")
            plt.grid(True)
            plt.tight_layout()
            plt.show()


# --- Liaison et affichage ---
widgets.interactive_output(
    on_puit_change,
    {
      'selected_puit': dropdown,
      'show_vk': vk_toggle,
      'show_summary': summary_toggle,
        'show_ab': ab_toggle,
        'class_ab': classab_toggle,
        'vkclass': venclass_toggle
    }
)

display(widgets.VBox([dropdown, summary_toggle, vk_toggle,ab_toggle,classab_toggle,venclass_toggle, output]))